# Creating structures in pyiron

This section gives a brief introduction about some of the tools available in pyiron to construct atomic structures. 

For the sake of compatibility, our structure class is written to be compatible with the popular Atomistic Simulation Environment package ([ASE](https://wiki.fysik.dtu.dk/ase/)). This makes it possible to use routines from ASE to help set-up structures.

Furthermore, pyiron uses the [NGLview](http://nglviewer.org/nglview/latest/api.html) package to visualize the structures and trajectories interactively in 3D using NGLview-widgets.

As preparation for the following discussion we import a few python libraries

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pylab as plt

and create a pyiron project named 'structures':

In [2]:
from pyiron.project import Project
pr = Project(path='structures')

## Bulk crystals

In this section we discuss various possibilities to create bulk crystal structures.

### Using `create_structure()`

The simplest way to generate simple crystal structures is using the inbuilt `create_structure()` function specifying the element symbol, Bravais basis and the lattice constant(s)

Note: The output gives a cubic cell rather than the smallest non-orthogonal unit cell.

In [3]:
structure = pr.create_structure('Al', 
                            bravais_basis='fcc', 
                            lattice_constant=4.05)

To plot the structure interactively in 3D simply use:

In [4]:
structure.plot3d()

_ColormakerRegistry()

NGLWidget()

### Using `create_ase_bulk()`

Another convenient way to set up structures is using the `create_ase_bulk()` function which is built on top of the ASE build package for [bulk crystals](https://wiki.fysik.dtu.dk/ase/ase/build/build.html#ase.build.bulk). This function returns an object which is of the pyiron structure object type.

**Example:** fcc bulk aluminum in a cubic cell

In [5]:
structure = pr.create_ase_bulk('Al', cubic=True)
structure.plot3d()

NGLWidget()

**Example:** wurtzite GaN in a 3x3x3 repeated orthorhombic cell.

Note: 
- In contrast to new_structure = structure.repeat() which creates a new object, set_repeat() modifies the existing structure object.
- Setting `spacefill=False` in the `plot3d()` method changes the atomic structure style to "ball and stick".

In [6]:
structure = pr.create_ase_bulk('AlN', 
                           crystalstructure='wurtzite', 
                           a=3.5, orthorhombic=True)
structure.set_repeat([3,3,3])
structure.plot3d(spacefill=False)

NGLWidget()

## Creating surfaces (using ASE)

Surfaces can be created using the `create_surface()` function which is also built on top of the ASE build package for [surfaces](https://wiki.fysik.dtu.dk/ase/_modules/ase/build/surface.html)

**Example:** Creating a 3x4 fcc Al(111) surface with 4 layers and a vacuum of 10 Ångström

In [7]:
Al_111 = pr.create_surface("Al", surface_type="fcc111", 
                           size=(3, 4, 4), vacuum=10, orthogonal=True)
Al_111.plot3d()

NGLWidget()

## Creating structures without importing the project class

In all the examples shown above, the structures are create from the pyiron `Project` object. It is also possible to do this without importing/initializing this object. For this the appropriate imports must be made.

In [8]:
from pyiron import create_ase_bulk, create_surface

In [9]:
structure = create_ase_bulk('AlN', 
                            crystalstructure='wurtzite', 
                            a=3.5, orthorhombic=True)
structure.set_repeat([3,3,3])
structure.plot3d(spacefill=False)

NGLWidget()

In [10]:
Al_111 = create_surface("Al", surface_type="fcc111", 
                        size=(3, 4, 4), vacuum=10, orthogonal=True)
Al_111.plot3d()

NGLWidget()

### Using the ASE spacegroup class

In [11]:
from ase.spacegroup import crystal
from pyiron import ase_to_pyiron

a = 9.04
skutterudite = crystal(('Co', 'Sb'),
                       basis=[(0.25, 0.25, 0.25), (0.0, 0.335, 0.158)],
                       spacegroup=204,
                       cellpar=[a, a, a, 90, 90, 90])
skutterudite = ase_to_pyiron(skutterudite)

In [12]:
skutterudite.plot3d()

NGLWidget()

## Accessing the properties of the structure object

Using the bulk aluminum fcc example from before the structure object can be created by

In [13]:
structure = pr.create_ase_bulk('Al', cubic=True)

A summary of the information about the structure is given by using

In [14]:
print(structure)

Al: [0. 0. 0.]
Al: [0.    2.025 2.025]
Al: [2.025 0.    2.025]
Al: [2.025 2.025 0.   ]
pbc: [ True  True  True]
cell: 
[[4.05 0.   0.  ]
 [0.   4.05 0.  ]
 [0.   0.   4.05]]



The cell vectors of the structure object can be accessed and edited through

In [15]:
structure.cell

array([[4.05, 0.  , 0.  ],
       [0.  , 4.05, 0.  ],
       [0.  , 0.  , 4.05]])

The positions of the atoms in the structure object can be accessed and edited through

In [16]:
structure.positions

array([[0.   , 0.   , 0.   ],
       [0.   , 2.025, 2.025],
       [2.025, 0.   , 2.025],
       [2.025, 2.025, 0.   ]])

## Point defects

### Creating a single vacancy

We start by setting up a 4x4x4 supercell

In [17]:
structure = pr.create_ase_bulk('Al', cubic=True)
structure.set_repeat([4,4,4])

To create the vacancy at position index "0" simply use:

In [18]:
del structure[0]

To plot the structure that now contains a vacancy run:

In [19]:
structure.plot3d()

NGLWidget()

### Creating multiple vacancies

In [20]:
# First create a 4x4x4 supercell
structure = pr.create_ase_bulk('Al', cubic=True)
structure.set_repeat([4,4,4])
print('Number of atoms in the repeat unit: ',structure.get_number_of_atoms())

Number of atoms in the repeat unit:  256


The `del` command works for passing a list of indices to the structure object. For example, a random set of n$_{\text{vac}}$ vacancies can be created by using

In [21]:
# Generate a list of indices for the vacancies
n_vac = 24
vac_ind_lst = np.random.permutation(len(structure))[:n_vac]

# Remove atoms according to the "vac_ind_lst"
del structure[vac_ind_lst]

In [22]:
# Visualize the structure
print('Number of atoms in the repeat unit: ',structure.get_number_of_atoms())
structure.plot3d()

Number of atoms in the repeat unit:  232


NGLWidget()

### Random substitutial alloys

In [23]:
# Create a 4x4x4 supercell
structure = pr.create_ase_bulk('Al', cubic=True)
structure.set_repeat([4,4,4])

Substitutional atoms can be defined by changing the atomic species accessed through its position index.

Here, we set $n_{\text{sub}}$ magnesium substitutional atoms at random positions

In [24]:
n_sub = 24
structure[np.random.permutation(len(structure))[:n_sub]] = 'Mg'

In [25]:
# Visualize the structure and print some additional information about the structure
print('Number of atoms in the repeat unit: ',structure.get_number_of_atoms())
print('Chemical formula: ',structure.get_chemical_formula())
structure.plot3d()

Number of atoms in the repeat unit:  256
Chemical formula:  Al232Mg24


NGLWidget()

## Explicit definition of the structure

You can also set-up structures through the explicit input of the cell parameters and positions

In [26]:
cell = 10.0 * np.eye(3) # Specifying the cell dimensions
positions = [[0.25, 0.25, 0.25], [0.75, 0.75, 0.75]]
elements = ['O', 'O']

# Now use the Atoms class to create the instance.
O_dimer = pr.create_atoms(elements=elements, scaled_positions=positions, cell=cell)

O_dimer.plot3d()

NGLWidget()

## Importing from cif/other file formats

Parsers from ASE can be used to import structures from other formats. In this example, we will download and import a Nepheline structure from the [Crystallography Open Database (COD)](http://www.crystallography.net/cod/index.php)

In [27]:
# The COD structures can be accessed through their unique COD identifier
filename = '1008753.cif'
url = 'http://www.crystallography.net/cod/{}'.format(filename)

In [28]:
# Download and save the structure file locally
import urllib
urllib.request.urlretrieve(url=url, filename='strucs.'+filename);

In [29]:
# Using ase parsers to read the structure and then convert to a pyiron instance
import ase
from pyiron import ase_to_pyiron

structure = ase_to_pyiron(ase.io.read(filename='strucs.'+filename,
                                      format='cif'))

/home/surendralal/miniconda3/envs/pyiron_workshop/lib/python3.7/site-packages/ase/io/cif.py:375: UserWarning: crystal system 'hexagonal' is not interpreted for space group Spacegroup(173, setting=1). This may result in wrong setting!
  setting_name, spacegroup))


In [30]:
structure.plot3d()

NGLWidget()